# Process hourly data to daily

This notebook is to process the base variables that will be used as predictors in the energy demand model, or will be used in computations of further indices.

- Daily mean temperature - Done.
- Maximum daily T - Done
- Minimum daily T - Done
- Number of sun hours / Cloud cover (use solar radiation as proxy?)
- Solar radiation - Done
- Wind speed - Done
- Rainfall - Done
- Relative humidity (daily average and/or overnight?) - Done (daily avg only)
- Specific humidity - Done
- Dew point (average daily)
- Dew point T (maximum hour of the day)

We already have some variables computed from the energy/climate modes project: https://github.com/dougrichardson/energy_climate_modes/blob/main/1_hourly_to_daily.ipynb

In [1]:
from dask.distributed import Client,LocalCluster
from dask_jobqueue import PBSCluster

In [18]:
# One node on Gadi has 48 cores - try and use up a full node before going to multiple nodes (jobs)

walltime = "01:00:00"
cores = 24
memory = str(4 * cores) + "GB"

cluster = PBSCluster(walltime=str(walltime), cores=cores, memory=str(memory), processes=cores,
                     job_extra_directives=["-q normal",
                                           "-P w42",
                                           "-l ncpus="+str(cores),
                                           "-l mem="+str(memory),
                                           "-l storage=gdata/w42+gdata/rt52"],
                     local_directory="$TMPDIR",
                     job_directives_skip=["select"])

In [19]:
cluster.scale(jobs=1)
client = Client(cluster)

In [20]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.121.49:37529,Workers: 0
Dashboard: /proxy/8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [5]:
import xarray as xr
import numpy as np

In [6]:
root_path = "/g/data/rt52/era5/single-levels/reanalysis/"
write_path = "/g/data/w42/dr6273/work/data/era5/"

In [7]:
years = range(1959, 2023)

The demand data has been converted to UTC, so we can use UTC for ERA5, too.

In [8]:
desired_first_hour = 0 # Set the desired first hour of the day

In [9]:
def hourly_to_daily(aggregate_function, variable, year, first_hour, data_path=root_path):
    """
    Compute 24-hour aggregates from hourly data for a given year.
    
    aggregate_function: function to aggregate hourly data (e.g. mean, max)
    variable: name of variable to process
    year:  year to process
    first_hour: desired first hour from which to compute 24-hour aggregations
    data_path: path to hourly data
    """
    # Open all hours in the year (~33 GB)
    hourly = xr.open_mfdataset(
        data_path + variable + "/" + str(year) + "/*.nc",
        chunks={"time": 24}
    )

    # Start the aggregation on the desired hour (e.g. 0000)
    data_first_hour = hourly["time"].dt.hour.item(0)
    desired_start_index = (first_hour - data_first_hour) % 24
    hourly = hourly.isel(time=range(desired_start_index, len(hourly["time"])))

    # Resample to daily means
    daily = aggregate_function(hourly)

    # Re-assign time to be 0000 hour of each day
    daily = daily.assign_coords({"time": daily["time"].dt.floor("1D")})
    
    return daily

In [10]:
def write_daily(daily, variable, year, first_hour, write_path=write_path, encoding_name=None):
    """
    Write daily data to file
    
    daily: DataArray to be written
    variable: name of variable
    year: year being written
    data_path: path to hourly data
    encoding_name: name of variable to encode. Usually the same as variable.
    """
    # Chunk
    daily = daily.chunk({"time": 24})

    # Write to netcdf
    if isinstance(encoding_name, str):
        name = encoding_name
    else:
        name = variable
        
    encoding = {
        name: {"dtype": "float32"}
    }
    daily.to_netcdf(
        write_path + variable + "/daily/" + variable + "_era5_daily_" + str(year) + ".nc",
        mode="w",
        encoding=encoding
    )

In [11]:
def write_by_year(aggregate_function, variable, years, first_hour, data_path, write_path, encoding_name=None):
    """
    Compute 24-hour aggregations from hourly data and write to file for each year.
    
    aggregate_function: function to aggregate hourly data (e.g. mean, max)
    variable: name of variable to process
    years: range of years to process
    first_hour: desired first hour from which to compute 24-hour aggregations
    data_path: path to hourly data
    encoding_name: name of variable to encode. Usually the same as variable.
    """
    for year in years:
        # print(year)

        daily = hourly_to_daily(aggregate_function, variable, year, first_hour, data_path=data_path)
        
        write_daily(daily, variable, year, first_hour, write_path=write_path, encoding_name=encoding_name)

In [12]:
def daily_mean(da):
    """
    24-hour mean of da.
    """
    return da.coarsen(time=24, boundary="trim").mean(keep_attrs=True)

# Solar radiation - available from other project

In [10]:
# write_by_year(daily_mean, "msdwswrf", range(2022, 2023), desired_first_hour, root_path, write_path)

# 10 metre windspeed - available from other project

Wind speed needs different treatment, because we need daily values of u and v, then compute W.

In [15]:
# def windspeed(u, v):
#     """
#     Compute windspeed from u and v
    
#     u: array of zonal wind
#     v: array of meridional wind
#     """
#     return np.sqrt(u ** 2 + v ** 2)

In [16]:
# def windspeed_write_by_year(aggregate_function, u_name, v_name, w_name, years, first_hour, data_path, write_path):
#     """
#     Compute 24-hour means from hourly data and write to file for each year.
    
#     aggregate_function: function to aggregate hourly data (e.g. mean, max)
#     u_name: name of u variable to process (e.g. 10u)
#     v_name: name of v variable to process
#     w_name: name of output wind speed
#     years: range of years to process
#     first_hour: desired first hour from which to compute 24-hour means
#     data_path: path to hourly data
#     """
#     for year in years:
#         print(year)

#         # Get daily means of u and v
#         u_daily = hourly_to_daily(aggregate_function, u_name, year, first_hour, data_path=data_path)
#         v_daily = hourly_to_daily(aggregate_function, v_name, year, first_hour, data_path=data_path)
        
#         # Data variables have u/v/w before the level e.g. u10 instead of 10u
#         # This is different to how the directories are structured and is annoying
#         # We set some different variable names for the arrays.
#         da_u_name = u_name[-1] + u_name[:-1]
#         da_v_name = v_name[-1] + v_name[:-1]
#         da_w_name = w_name[-1] + w_name[:-1]
        
#         # Windspeed
#         w_daily = windspeed(
#             u_daily.rename({da_u_name: da_w_name}),
#             v_daily.rename({da_v_name: da_w_name})
#         )
        
#         # Write to file. Note we specify encoding name here
#         write_daily(w_daily, w_name, year, first_hour, write_path=write_path, encoding_name=da_w_name)

In [17]:
# windspeed_write_by_year(daily_mean, '10u', '10v', "10w", range(1994, 2023), desired_first_hour, root_path, write_path)

# 2 metre daily mean temperature - available from other project

In [18]:
# write_by_year(daily_mean, "2t", years, desired_first_hour, root_path, write_path, encoding_name='t2m')

# 2 metre maximum daily temperature - available from other project

In [21]:
def daily_max(da):
    """
    24-hour max of da.
    """
    return da.coarsen(time=24, boundary="trim").max(keep_attrs=True)

In [23]:
def tmax_write_by_year(variable, years, first_hour, data_path, write_path):
    """
    Compute 24-hour maximum temperature from hourly data and write to file for each year.
    
    variable: name of variable to process
    years: range of years to process
    first_hour: desired first hour from which to compute 24-hour aggregations
    data_path: path to hourly data
    """
    for year in years:
        # print(year)

        daily = hourly_to_daily(daily_max, variable, year, first_hour, data_path=data_path)
        
        write_daily(daily, "2tmax", year, first_hour, write_path=write_path, encoding_name='mx2t')

In [24]:
tmax_write_by_year("mx2t", years, desired_first_hour, root_path, write_path)

# 2 metre minimum daily temperature

In [22]:
def daily_min(da):
    """
    24-hour min of da.
    """
    return da.coarsen(time=24, boundary="trim").min(keep_attrs=True)

In [26]:
def tmin_write_by_year(variable, years, first_hour, data_path, write_path):
    """
    Compute 24-hour minimum temperature from hourly data and write to file for each year.
    
    variable: name of variable to process
    years: range of years to process
    first_hour: desired first hour from which to compute 24-hour aggregations
    data_path: path to hourly data
    """
    for year in years:
        # print(year)

        daily = hourly_to_daily(daily_min, variable, year, first_hour, data_path=data_path)
        
        write_daily(daily, "2tmin", year, first_hour, write_path=write_path, encoding_name='mn2t')

In [27]:
tmin_write_by_year("mn2t", years, desired_first_hour, root_path, write_path)

# 2 metre daily mean dew point temperature

In [15]:
write_by_year(daily_mean, "2d", years, desired_first_hour, root_path, write_path, encoding_name='d2m')

# 2 metre maximum daily dew point temperature

No maximum dew point variable, so take the hour that is the maximum of the hourly means per day.

In [23]:
def td_max_write_by_year(variable, years, first_hour, data_path, write_path):
    """
    Compute 24-hour maximum dew-point temperature from hourly data and write to file for each year.
    
    variable: name of variable to process
    years: range of years to process
    first_hour: desired first hour from which to compute 24-hour aggregations
    data_path: path to hourly data
    """
    for year in years:
        # print(year)

        daily = hourly_to_daily(daily_max, variable, year, first_hour, data_path=data_path)
        
        write_daily(daily, "2dmax", year, first_hour, write_path=write_path, encoding_name='d2m')

In [ ]:
td_max_write_by_year("2d", years, desired_first_hour, root_path, write_path)

# 2 metre minimum daily dew point temperature

No minimum dew point variable, so take the hour that is the minimum of the hourly means per day.

In [ ]:
def td_min_write_by_year(variable, years, first_hour, data_path, write_path):
    """
    Compute 24-hour minimum dew-point temperature from hourly data and write to file for each year.
    
    variable: name of variable to process
    years: range of years to process
    first_hour: desired first hour from which to compute 24-hour aggregations
    data_path: path to hourly data
    """
    for year in years:
        # print(year)

        daily = hourly_to_daily(daily_min, variable, year, first_hour, data_path=data_path)
        
        write_daily(daily, "2dmin", year, first_hour, write_path=write_path, encoding_name='d2m')

In [ ]:
td_min_write_by_year("2d", years, desired_first_hour, root_path, write_path)

# Precipitation

We use the mean total precipitation rate (kg/m^2/s), because it is available on NCI for ERA5 (not only ERA5 Land, as total precipitation is).

The mean rate hydrological parameters (e.g. the "Mean total precipitation rate") have units of "kg m-2 s-1", which are equivalent to "mm s-1". They can be multiplied by 86400 seconds (24 hours) to convert to kg m-2 day-1 or mm day-1.

Note that this conversion is *not done here*.

https://confluence.ecmwf.int/display/CKB/ERA5%3A+data+documentation#ERA5:datadocumentation-Parameterlistings

In [63]:
write_by_year(daily_mean, "mtpr", years, desired_first_hour, root_path, write_path)

# Relative humidity

Formula from Bolton 1980: https://journals.ametsoc.org/view/journals/mwre/108/7/1520-0493_1980_108_1046_tcoept_2_0_co_2.xml?tab_body=pdf

As used in `metpy` package: https://github.com/Unidata/MetPy/blob/main/src/metpy/calc/thermo.py#L1278

Relative humidity is

$$ \psi = \frac{e}{e_s} \times 100 $$
    
where $e$ is the water vapour pressure, dependent on the dew-point temperature $T_d [^\circ$C], and $e_s$ is the saturation vapor pressure, dependent on temperature, $T [^\circ$C]. The following formula can be used to calculate both $e$ and $e_s$ by using $T^* = T_d$ or $T^* = T$ as the input, respectively:

$$ e^* = 6.112 \exp \left( \frac{17.67T^*}{T^* + 243.5} \right) $$

In [47]:
def saturation_vapour_pressure(T):
    """
    Calculates the saturation vapour pressure if T is air temperature,
    or the vapour pressure if T is dew-point temperature. Units of T
    should be degrees Celsius.
    """
    return 6.112 * np.exp( (17.67 * T) / (T + 243.5) )

In [48]:
def relative_humidity(T, T_d):
    """
    Calculates relative humidity from temperature and dew-point temperature.
    Units should be degrees Celsius.
    """
    e = saturation_vapour_pressure(T_d)
    e_s = saturation_vapour_pressure(T)
    return 100 * (e / e_s)

In [49]:
def rel_hum_write_by_year(aggregate_function, T_name, Td_name, phi_name, years, first_hour, data_path, write_path):
    """
    Compute 24-hour means of relative humidity from hourly temperature and dew-point data and write to file for each year.
    
    aggregate_function: function to aggregate hourly data (e.g. mean, max)
    T_name: name of temperature variable to process
    Td_name: name of dew-point temperature variable to process
    phi_name: name of output relative humidity
    years: range of years to process
    first_hour: desired first hour from which to compute 24-hour means
    data_path: path to hourly data
    """
    for year in years:
        # Get daily means of T and Td
        T_daily = hourly_to_daily(aggregate_function, T_name, year, first_hour, data_path=data_path)
        Td_daily = hourly_to_daily(aggregate_function, Td_name, year, first_hour, data_path=data_path)
        
        # Convert K to deg C
        T_daily = T_daily - 273.15
        Td_daily = Td_daily - 273.15
        
        # Rel. hum.
        phi_daily = relative_humidity(
            T_daily.rename({"t2m": phi_name}), # Data variables are t2m/d2m, not 2t/2d
            Td_daily.rename({"d2m": phi_name})
        )
        
        # Write to file. Note we specify encoding name here
        write_daily(phi_daily, phi_name, year, first_hour, write_path=write_path, encoding_name=phi_name)

In [51]:
rel_hum_write_by_year(daily_mean, "2t", "2d", "rh", years, desired_first_hour, root_path, write_path)

# Specific humidity

Not available at the surface in ERA5, so we use the following formula:
$$ q = \frac{0.622e}{p - 0.378e}, $$
where $e$ is the water vapour pressure as defined above (as a function of $T_{d}$) and $p$ is the surface pressure (mb).

From: https://archive.eol.ucar.edu/projects/ceop/dm/documents/refdata_report/eqns.html

In [52]:
def specific_humidity(T_d, p):
    """
    Calculates specific humidity from dew-point temperature and surface pressure.
    Units should be degrees Celsius and millibars.
    """
    e = saturation_vapour_pressure(T_d)
    return (0.622 * e) / (p - 0.378 * e)

In [53]:
def specific_hum_write_by_year(aggregate_function, Td_name, sp_name, q_name, years, first_hour, data_path, write_path):
    """
    Compute 24-hour means of specific humidity from hourly dew-point temperature and surface pressure data and write to file for each year.
    
    aggregate_function: function to aggregate hourly data (e.g. mean, max)
    Td_name: name of dew-point temperature variable to process
    sp_name: name of surface pressure variable to process
    q_name: name of output specific humidity
    years: range of years to process
    first_hour: desired first hour from which to compute 24-hour means
    data_path: path to hourly data
    """
    for year in years:
        # Get daily means of Td and sp
        Td_daily = hourly_to_daily(aggregate_function, Td_name, year, first_hour, data_path=data_path)
        sp_daily = hourly_to_daily(aggregate_function, sp_name, year, first_hour, data_path=data_path)
        
        # Convert pressure from Pa to mb and K to deg C
        Td_daily = Td_daily - 273.15
        sp_daily = sp_daily / 100
        
        # Spec. hum.
        q_daily = specific_humidity(
            Td_daily.rename({"d2m": q_name}),
            sp_daily.rename({"sp": q_name})
        )
        # Convert to g/kg
        q_daily = q_daily * 1000
        
        # Write to file. Note we specify encoding name here
        write_daily(q_daily, q_name, year, first_hour, write_path=write_path, encoding_name=q_name)

In [54]:
specific_hum_write_by_year(daily_mean, "2d", "sp", "q", years, desired_first_hour, root_path, write_path)

# Close cluster

In [55]:
client.close()
cluster.close()